In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import XL_BOMD_MD, Molecular_Dynamics_Basic, XL_BOMD_LR_MD, Molecular_Dynamics_Langevin
from seqm.seqm_functions.read_xyz import read_xyz
from torch.profiler import profile, record_function, ProfilerActivity
from seqm.ElectronicStructure import Electronic_Structure
import numpy as np
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device

device(type='cuda')

In [47]:
dt = 0.7
Temperature = 600.0
Nsteps = 5000

## BOMD single pr

In [40]:
%%time
torch.set_default_dtype(torch.float32)
torch.manual_seed(0)
### create molecule object:
files = ['C36.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]

coordinates = torch.tensor(coordinates, dtype=torch.get_default_dtype(), device=device)[:]
const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'BOMD_sing'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=Temperature, timestep=dt, output=output).to(device)
md.initialize_velocity(molecule, vel_com=True )

_ = md.run(molecule, Nsteps, reuse_P=True, remove_com=[True, 1], Info_log=True)



Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   509.90   4.284103e+00 6.957062e+00 1.124117e+01 || 
     2   450.37   3.783988e+00 7.453156e+00 1.123714e+01 || 
     3   381.68   3.206819e+00 8.089874e+00 1.129669e+01 || 
     4   326.74   2.745244e+00 8.550903e+00 1.129615e+01 || 
     5   291.65   2.450415e+00 8.824249e+00 1.127466e+01 || 
     6   274.20   2.303764e+00 9.000122e+00 1.130389e+01 || 
     7   267.02   2.243509e+00 9.074249e+00 1.131776e+01 || 
     8   258.55   2.172299e+00 9.117218e+00 1.128952e+01 || 
     9   240.63   2.021733e+00 9.230499e+00 1.125223e+01 || 
    10   223.44   1.877301e+00 9.385376e+00 1.126268e+01 || 
    11   233.20   1.959347e+00 9.351593e+00 1.131094e+01 || 
    12   278.19   2.337309e+00 8.937531e+00 1.127484e+01 || 
    13   332.11   2.790355e+00 8.475220e+00 1.126558e+01 || 
    14   358.97   3.016033e+00 8.256378e+00 1.127241e+01 || 
    15   345.49   2.902737e+00 8.346344e+00 1.124908e+01 ||

## BOMD double pr

In [51]:
%%time
torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
### create molecule object:
files = ['coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]

coordinates = torch.tensor(coordinates, dtype=torch.get_default_dtype(), device=device)[:]
const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'BOMD_doub'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=Temperature, timestep=dt, output=output).to(device)
md.initialize_velocity(molecule, vel_com=True )

_ = md.run(molecule, Nsteps, reuse_P=True, remove_com=[True, 1], Info_log=True)



Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   600.98   2.796566e+00 4.381483e+00 7.178049e+00 || 
     2   534.92   2.489165e+00 4.700827e+00 7.189992e+00 || 
     3   467.13   2.173736e+00 5.024016e+00 7.197752e+00 || 
     4   422.17   1.964516e+00 5.232593e+00 7.197109e+00 || 
     5   399.97   1.861203e+00 5.330880e+00 7.192082e+00 || 
     6   385.82   1.795368e+00 5.392949e+00 7.188317e+00 || 
     7   366.76   1.706689e+00 5.480450e+00 7.187139e+00 || 
     8   337.55   1.570744e+00 5.616413e+00 7.187157e+00 || 
     9   298.56   1.389325e+00 5.799086e+00 7.188411e+00 || 
    10   256.27   1.192498e+00 6.000318e+00 7.192817e+00 || 
    11   226.78   1.055268e+00 6.144099e+00 7.199367e+00 || 
    12   229.81   1.069409e+00 6.131354e+00 7.200762e+00 || 
    13   267.78   1.246054e+00 5.945856e+00 7.191910e+00 || 
    14   313.60   1.459308e+00 5.720331e+00 7.179639e+00 || 
    15   330.47   1.537792e+00 5.637418e+00 7.175210e+00 ||

## XL single

In [42]:
%%time
torch.set_default_dtype(torch.float32)
torch.manual_seed(0)
### create molecule object:
files = ['C36.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, dtype=torch.get_default_dtype(), device=device)[:]
const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'XL_sing'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
md = XL_BOMD_LR_MD(max_rank=3, err_threshold=0.0, T_el=1500, k=6,
                   seqm_parameters=seqm_parameters, Temp=Temperature, timestep=dt, output=output).to(device)
md.initialize_velocity(molecule, vel_com=True )

_ = md.run(molecule, Nsteps, remove_com=[True, 1], Info_log=True)




Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   505.98   4.251230e+00 6.996123e+00 1.124735e+01 || 
     2   441.18   3.706792e+00 7.480499e+00 1.118729e+01 || 
     3   373.44   3.137576e+00 8.093781e+00 1.123136e+01 || 
     4   324.34   2.725096e+00 8.628937e+00 1.135403e+01 || 
     5   293.13   2.462879e+00 8.765656e+00 1.122853e+01 || 
     6   275.04   2.310820e+00 9.053345e+00 1.136417e+01 || 
     7   265.61   2.231587e+00 9.027466e+00 1.125905e+01 || 
     8   255.54   2.146981e+00 9.117218e+00 1.126420e+01 || 
     9   237.82   1.998111e+00 9.248657e+00 1.124677e+01 || 
    10   222.55   1.869801e+00 9.453247e+00 1.132305e+01 || 
    11   232.80   1.955970e+00 9.375031e+00 1.133100e+01 || 
    12   276.68   2.324658e+00 8.980499e+00 1.130516e+01 || 
    13   330.41   2.776107e+00 8.496124e+00 1.127223e+01 || 
    14   358.10   3.008678e+00 8.238312e+00 1.124699e+01 || 
    15   345.59   2.903653e+00 8.36721

## XL double

In [52]:
%%time
torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
### create molecule object:
files = ['coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, dtype=torch.get_default_dtype(), device=device)[:]
const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'XL_doub'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
md = XL_BOMD_LR_MD(max_rank=3, err_threshold=0.0, T_el=1500, k=6,
                   seqm_parameters=seqm_parameters, Temp=Temperature, timestep=dt, output=output).to(device)
md.initialize_velocity(molecule, vel_com=True )

_ = md.run(molecule, Nsteps, remove_com=[True, 1], Info_log=True)




Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   597.54   2.780585e+00 4.397792e+00 7.178377e+00 || 
     2   526.41   2.449590e+00 4.705973e+00 7.155563e+00 || 
     3   458.66   2.134296e+00 5.023253e+00 7.157549e+00 || 
     4   417.84   1.944360e+00 5.241444e+00 7.185804e+00 || 
     5   399.09   1.857121e+00 5.328383e+00 7.185504e+00 || 
     6   385.29   1.792907e+00 5.392292e+00 7.185200e+00 || 
     7   364.56   1.696451e+00 5.480248e+00 7.176699e+00 || 
     8   334.06   1.554517e+00 5.608796e+00 7.163313e+00 || 
     9   295.69   1.375975e+00 5.794065e+00 7.170040e+00 || 
    10   255.28   1.187888e+00 5.995236e+00 7.183124e+00 || 
    11   226.54   1.054189e+00 6.137155e+00 7.191344e+00 || 
    12   228.09   1.061385e+00 6.130964e+00 7.192349e+00 || 
    13   264.20   1.229436e+00 5.949124e+00 7.178561e+00 || 
    14   309.84   1.441805e+00 5.720289e+00 7.162094e+00 || 
    15   327.72   1.525008e+00 5.63651

## XL old single

In [44]:
%%time
torch.set_default_dtype(torch.float32)
torch.manual_seed(0)
### create molecule object:
files = ['C36.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, dtype=torch.get_default_dtype(), device=device)[:]
const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'XL_old_sing'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
md = XL_BOMD_MD(k=6,
                   seqm_parameters=seqm_parameters, Temp=Temperature, timestep=dt, output=output).to(device)
md.initialize_velocity(molecule, vel_com=True )

_ = md.run(molecule, Nsteps, remove_com=[True, 1], Info_log=True)




Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   505.79   4.249576e+00 7.029907e+00 1.127948e+01 || 
     2   440.06   3.697374e+00 7.503937e+00 1.120131e+01 || 
     3   370.45   3.112521e+00 8.045532e+00 1.115805e+01 || 
     4   320.50   2.692846e+00 8.562531e+00 1.125538e+01 || 
     5   290.66   2.442061e+00 8.847687e+00 1.128975e+01 || 
     6   274.99   2.310459e+00 8.988312e+00 1.129877e+01 || 
     7   266.92   2.242653e+00 8.980499e+00 1.122315e+01 || 
     8   256.12   2.151904e+00 9.105499e+00 1.125740e+01 || 
     9   236.35   1.985752e+00 9.250031e+00 1.123578e+01 || 
    10   219.66   1.845603e+00 9.394562e+00 1.124016e+01 || 
    11   230.43   1.936039e+00 9.398560e+00 1.133460e+01 || 
    12   275.37   2.313617e+00 8.932251e+00 1.124587e+01 || 
    13   329.14   2.765402e+00 8.484406e+00 1.124981e+01 || 
    14   356.85   2.998225e+00 8.195343e+00 1.119357e+01 || 
    15   345.15   2.899890e+00 8.359497e+00 1.125939e+01 ||

## XL old double

In [53]:
%%time
torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
### create molecule object:
files = ['coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, dtype=torch.get_default_dtype(), device=device)[:]
const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'XL_old_doub'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
md = XL_BOMD_MD(k=6,
                   seqm_parameters=seqm_parameters, Temp=Temperature, timestep=dt, output=output).to(device)
md.initialize_velocity(molecule, vel_com=True )

_ = md.run(molecule, Nsteps, remove_com=[True, 1], Info_log=True)




Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   597.54   2.780585e+00 4.397792e+00 7.178377e+00 || 
     2   525.52   2.445435e+00 4.713245e+00 7.158681e+00 || 
     3   455.68   2.120433e+00 5.020570e+00 7.141003e+00 || 
     4   413.52   1.924235e+00 5.237973e+00 7.162208e+00 || 
     5   395.95   1.842487e+00 5.336924e+00 7.179411e+00 || 
     6   385.22   1.792585e+00 5.388760e+00 7.181344e+00 || 
     7   366.84   1.707040e+00 5.475511e+00 7.182551e+00 || 
     8   335.90   1.563059e+00 5.613805e+00 7.176865e+00 || 
     9   294.85   1.372064e+00 5.791324e+00 7.163387e+00 || 
    10   252.27   1.173922e+00 5.989566e+00 7.163488e+00 || 
    11   224.08   1.042706e+00 6.141281e+00 7.183987e+00 || 
    12   227.78   1.059942e+00 6.131615e+00 7.191557e+00 || 
    13   264.90   1.232677e+00 5.946101e+00 7.178778e+00 || 
    14   309.95   1.442292e+00 5.725017e+00 7.167309e+00 || 
    15   327.22   1.522657e+00 5.637475e+00 7.160132e+00 ||